In [6]:
from datetime import datetime, date, timedelta
import numpy as np
import xarray as xr

import pandas as pd

import sys
sys.path.append('/home/ascherrmann/scripts/')
import helper
import argparse
import pickle
import os

In [14]:
######################################################################################################
## This cell is just a test

path_trackCYC = f'/net/thermo/atmosdyn/atroman/PAC1d/cdf/TRACKED_CYCLONES'


# d = np.loadtxt(path_trackCYC,skiprows=1)
d = pd.read_csv(path_trackCYC, skiprows=1, delim_whitespace=True, na_values='NA').values
IDs = np.unique(d[:,1])
# try:
# except IndexError:
#     IDs = np.unique(d[-1])
print(f'IDs from the TRACKED_CYCLONE file:\n{IDs}')
######################################################################################################

IDs from the TRACKED_CYCLONE file:
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]


---
## Create File with cyclone information of case study 17 

### Structure of d:
-  0 col:    Time
-  1 col:    ID
-  2 col:    LON
-  3 col:    LAT
-  4 col:    Distance
-  5 col:    Area
-  6 col:    Inpres
-  7 col:    Outpres
-  8 col:    baroclin
-  9 col:    meanth
- 10 col:    ctype
- 11 col:    bergeron
- 12 col:    bmaxdiff
---

In [134]:
## Initialise dictornary where important fields are saved
datastruct = dict()

labels = np.array([])
Gdates = np.array([])
data = dict()

LON = np.round(np.linspace(-180,180,901),1)
LAT = np.round(np.linspace(0,90,226),1)


add = 'CaseStudy-CYC-Apr17'

In [136]:
## LOAD DATA
################################# MODIFIED BY MAX ############################################################################
# - Load Cyclone Track file
path_trackCYC = f'/net/thermo/atmosdyn/atroman/PAC1d/cdf/TRACKED_CYCLONES'
d_all = pd.read_csv(path_trackCYC, skiprows=1, delim_whitespace=True, na_values='NA').values
IDs = np.unique(d_all[:,1])
print(f'IDs from the TRACKED_CYCLONE file:\n{IDs}')


IDs from the TRACKED_CYCLONE file:
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]


In [146]:
## Only select case-study cyclone: 
# time: 20170410_17   
# lon:  136.6
# lat:  47.2

## Filter for desired cyclone by lon/lat
d=d_all[(d_all[:,2]>=163.4) & (d_all[:,2]<=163.8) & (d_all[:,3]>=0) & (d_all[:,3]<=90)]

# row_min_slp = d[:,6].argmin() 
# d = d[row_min_slp]

d

array([[5.89720000e+05, 1.90000000e+01, 1.63600000e+02, 4.76000000e+01,
        2.63561711e+06, 2.87289299e+06, 9.65271851e+02, 1.00000000e+03,
        4.38835144e+01, 2.87381882e+02, 5.78752470e+00, 1.00000000e+00]])

In [149]:
ids = 19# d[1]
## cappear contains all indices of rows in d that have ids (one of the validation IDs)        
cappear, = np.where((d[:,1]==ids))  # This is just 0


In [150]:
cappear

array([0])

In [151]:
clat = []
clon = []
hourszeta = np.array([])
hourstoSLPmin = np.array([])
zetal = np.array([])
dates = np.array([])
zeta = np.zeros(len(cappear))
hours = np.zeros(len(cappear))


hoursslp = np.array([])         # MAX
slpl = np.array([])             # MAX
slp = np.zeros(len(cappear))    # MAX


### find id of minimum SLP in track file
slpminid = cappear[np.where(d[cappear,6] == np.min(d[cappear,6]))]   # NOTE: d[cappear,4] IS EQUAL TO d[:,4]   --> values of 5th column = "inpres"
slpminid = slpminid[-1]                                              # Extract number from array
hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0]) # Substract time from min SLP from other times to get seconds to/from SLP minimum


In [152]:
hourstoSLPmin

array([0.])

In [153]:
### step through hrs of the cyclon in trackfile
# for wr,u in enumerate(cappear):                   ## Not necessary, since cappear is only 0, i.e. wr == u == 0
wr,u = 0,0

tmp = d[u]

ft = date.toordinal(date(1950,1,1))
k = str(helper.datenum_to_datetime(ft+tmp[0]/24))
Date = k[0:4]+k[5:7]+k[8:10]+'_'+k[11:13]
k

'2017-04-10 16:00:00'

In [154]:
dates, Gdates = [],[]

# Load dataset with PS, SLP and Vorticity
dataset_1 = xr.open_dataset(f'/net/thermo/atmosdyn/atroman/PAC1d/cdf/2017/04/S{Date}', drop_variables=['PV','P','TH','THE','RH','PVRCONVT','PVRCONVM','PVRTURBT','PVRTURBM','PVRLS','PVRCOND','PVRSW','PVRLWH','PVRLWC','PVRDEP','PVREVC','PVREVR','PVRSUBI','PVRSUBS','PVRMELTI','PVRMELTS','PVRFRZ','PVRRIME','PVRBF'])
data[Date] = dataset_1

dates = np.append(dates,Date)
Gdates = np.append(Gdates,Date)

In [155]:
#hours since first identification
hours[wr] = tmp[0]
### keep adding lon to lon and lat to lat and use srtid, as it is correct with normal crosssections
CLONIDS, CLATIDS = helper.IFS_radial_ids_correct(200,LAT[np.where(LAT==np.round(tmp[3],1))[0][0]])
addlon = CLONIDS + np.where(LON==np.round(tmp[2],1))[0][0]
addlon[np.where((addlon-900)>0)] = addlon[np.where((addlon-900)>0)]-900

clat.append(CLATIDS.astype(int) + np.where(LAT==np.round(tmp[3],1))[0][0])
clon.append(addlon.astype(int))

In [156]:
### use last lon and lat entries [-1] of center
PS = data[Date].PS.values[0,0,clat[-1],clon[-1]]
for e in range(len(CLATIDS)):
    P = helper.modellevel_to_pressure(PS[e])
    I = np.where(abs(P-850)==np.min(abs(P-850)))[0]
    I = I[0].astype(int)
    zetal = np.append(zetal,data[Date].VORT.values[0,I,clat[-1][e],clon[-1][e]])

zeta[wr] = np.mean(zetal)

hours = hours - hours[np.where(zeta==np.max(zeta))]
hourszeta = np.append(hourszeta,hours)

In [157]:
MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]]]),1)
MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]]]),1)

In [159]:

#if time to mature stage is too short remove from data
# if((hours[0]<=(-2)) & (hours[-1]>=2)):        # ALEX
if((hours[0]<=(-0)) & (hours[-1]>=0)):          # MAX
    datastruct[ids] = dict()
    datastruct[ids]['clat'] = clat
    datastruct[ids]['clon'] = clon
    datastruct[ids]['zeta'] = zeta
    datastruct[ids]['hzeta'] = hourszeta
    datastruct[ids]['SLP'] = d[cappear,6]
    datastruct[ids]['hSLP'] = hourstoSLPmin
    datastruct[ids]['dates'] = dates
    datastruct[ids]['Matlat'] = MatureLat
    datastruct[ids]['Matlon'] = MatureLon
    # ### these are preidentified tropical
    # if(d[cappear[0],-3]==0):
    #         label=0
    # ### MED
    # else:
    #     for b,o in enumerate(LONM[:-1]):
    #         if( ((30-MatureLat)<=0) & ((LATM[b]-MatureLat)>=0) & ((o-MatureLon)<=0) & ((LONM[b+1]-MatureLon)>=0)):
    #                 label=2
    # ### ATL
    #         else:
    #             label=1
    datastruct[ids]['label'] = ids
    print('Saved cyclone ID:', ids)

else: 
    print(f'File not saved, becuae time to mature stage is not long enough: {hours[0]}, {hours[-1]} ')

PATH = '/net/helium/atmosdyn/freimax/data_msc/IFS-17/ctraj'
# Create the directory if it does not exist

f = open(f'{PATH}/{add}.txt','wb')
pickle.dump(datastruct,f)
f.close()

Saved cyclone ID: 19
